# Dataset Cleaning 

The main objectives of this notebook are: 
- Import dataset 
- Get familiar with the data 
- Determine if there are data quality issues 
- Resolve any data quality issues 

In [34]:
import polars as pl 
import polars.selectors as cs 
from google.cloud import bigquery
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import sys 
import os 

%load_ext autoreload
%autoreload 2

# Manually add path to read from another folder 
path2add = os.path.normpath(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir, "utils"))
)
if not (path2add in sys.path):
    sys.path.append(path2add)

from preprocessing import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [36]:
# Create client object
client = bigquery.Client()

In [37]:
QUERY = ("SELECT `date`, `visitStartTime`, `fullVisitorId`, `geoNetwork`.`subContinent`, `geoNetwork`.`country`,"
        "`geoNetwork`.`city`,`device`.`browser`, `device`.`operatingSystem`,`device`.`deviceCategory`,"
        "`trafficSource`.`source`, `h`.`item`.`transactionId`,`hp`.`v2ProductName`,`hp`.`v2ProductCategory` "
        "FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170731`, UNNEST(hits) AS h,"
        "UNNEST(h.product) AS hp;")

In [38]:
run_query = client.query(QUERY)

In [39]:
df = pl.from_arrow(run_query.to_arrow())

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [40]:
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [41]:
df.shape

(49512, 13)

In [42]:
df.columns

['date',
 'visitStartTime',
 'fullVisitorId',
 'subContinent',
 'country',
 'city',
 'browser',
 'operatingSystem',
 'deviceCategory',
 'source',
 'transactionId',
 'v2ProductName',
 'v2ProductCategory']

In [43]:
df.sample()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501508750,"""6433803844369135216""","""Northern America""","""United States""","""San Francisco""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",null,"""Basecamp Explorer Powerbank Fl…","""Home/Electronics/"""


In [44]:
df.describe()

statistic,date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,str,f64,str,str,str,str,str,str,str,str,str,str,str
"""count""","""49512""",49512.0,"""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""462""","""49512""","""49512"""
"""null_count""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""49050""","""0""","""0"""
"""mean""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,21502.167394,null,null,null,null,null,null,null,null,null,null,null
"""min""","""20170731""",1.5015e9,"""0002457163364254438""","""Australasia""","""Algeria""","""(not set)""","""Amazon Silk""","""(not set)""","""desktop""","""(direct)""","""ORD201707311786""","""1 oz Hand Sanitizer""","""${escCatTitle}"""
"""25%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""max""","""20170731""",1.5016e9,"""9997362993085245352""","""Western Europe""","""Vietnam""","""not available in demo dataset""","""YaBrowser""","""iOS""","""tablet""","""youtube.com""","""ORD201707312663""","""YouTube Youth Short Sleeve Tee…","""Wearables/Men's T-Shirts/"""


In [45]:
df.select(pl.all().is_null().sum())

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,49050,0,0


In [46]:
df.filter(pl.col('transactionId') != 'null')

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Blackout Cap""","""Headgear"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Women's Vintage Hero Te…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Men's Long & Lean Tee C…","""Apparel"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Google 25 oz Red Stainless Ste…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android 17oz Stainless Steel S…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Short Sleeve Her…","""Apparel"""


In [47]:
df_cleaned = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
df_cleaned.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,str,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [48]:
df_cleaned = df_cleaned.with_columns(pl.when(pl.col('transactionId') != 'null').then(1).otherwise(0).alias("transactionId"))
df_cleaned

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,i32,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""
…,…,…,…,…,…,…,…,…,…,…,…,…
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android 17oz Stainless Steel S…","""Drinkware"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android Men's Short Sleeve Her…","""Apparel"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android Men's Take Charge Shor…","""Apparel"""


# Encoding the df

In [49]:
encoded_df = encoding(df)
encoded_df

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklea

fullVisitorId,country,browser,transactionId,v2ProductName
str,i32,i32,i32,i32
"""186231215995844689""",2,0,1,44
"""186231215995844689""",2,0,1,44
"""0345672699449577691""",2,0,1,26
"""0345672699449577691""",2,0,1,75
"""0345672699449577691""",2,0,1,46
…,…,…,…,…
"""7483600664917507409""",2,0,1,23
"""7483600664917507409""",2,0,1,5
"""7483600664917507409""",2,0,1,10


In [50]:
user_profiles = encoded_df.group_by(pl.col('fullVisitorId')).agg(
    pl.col('country'), pl.col('browser'),
    pl.col('transactionId'), pl.col('v2ProductName')
)
user_profiles

fullVisitorId,country,browser,transactionId,v2ProductName
str,list[i32],list[i32],list[i32],list[i32]
"""4952396078130707023""","[2, 2]","[0, 0]","[1, 1]","[34, 34]"
"""813207140579582070""","[2, 2, … 2]","[0, 0, … 0]","[1, 1, … 1]","[75, 69, … 65]"
"""0047078955120420928""","[2, 2, … 2]","[0, 0, … 0]","[1, 1, … 1]","[97, 92, … 20]"
"""4408662176448539530""","[2, 2, … 2]","[0, 0, … 0]","[1, 1, … 1]","[79, 1, … 1]"
"""442359446750687328""","[2, 2, … 2]","[0, 0, … 0]","[1, 1, … 1]","[14, 55, … 55]"
…,…,…,…,…
"""5451348672261739783""","[2, 2]","[0, 0]","[1, 1]","[15, 15]"
"""4680625557979147601""","[2, 2]","[0, 0]","[1, 1]","[42, 42]"
"""1921325710867532207""","[2, 2, … 2]","[0, 0, … 0]","[1, 1, … 1]","[38, 69, … 97]"


In [75]:
user_profiles = user_profiles.filter(pl.col('fullVisitorId') != '7483600664917507409')

In [52]:
user_profiles_no_id = user_profiles.drop('fullVisitorId')
user_profiles_no_id = user_profiles_no_id.to_numpy()
user_profiles_no_id

array([[array([2, 2]), array([0, 0]), array([1, 1]), array([34, 34])],
       [array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
        array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
        array([75, 69, 68, 73, 65, 75, 69, 68, 73, 65])],
       [array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
        array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
        array([97, 92, 91, 96, 20, 97, 92, 91, 96, 20])],
       [array([2, 2, 2, 2]), array([0, 0, 0, 0]), array([1, 1, 1, 1]),
        array([79,  1, 79,  1])],
       [array([2, 2, 2, 2]), array([0, 0, 0, 0]), array([1, 1, 1, 1]),
        array([14, 55, 14, 55])],
       [array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
        array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
        array([40, 53, 97,  1, 32, 19, 40, 53, 97,  1, 32, 19])],
       [array([2, 2]), array([0, 0]), array([1, 1]), array([31, 31])],
       [array([2, 2, 2, 2

In [76]:
active_users = encoded_df.drop('fullVisitorId')
active_users = active_users.to_numpy()
active_users

array([[ 2,  0,  1, 44],
       [ 2,  0,  1, 44],
       [ 2,  0,  1, 26],
       ...,
       [ 2,  0,  1, 10],
       [ 2,  0,  1, 11],
       [ 2,  0,  1, 69]])

In [78]:
type(active_users[-1])

numpy.ndarray

In [79]:
similarities = calculate_similarity(user_profiles, user_profiles_no_id, active_users[-1])
similarities

{'4952396078130707023': 0.9994463821647174,
 '813207140579582070': 0.9999985949965018,
 '0047078955120420928': 0.9993473708718621,
 '4408662176448539530': 0.7187966887162289,
 '442359446750687328': 0.996020164630621,
 '1190814633327628141': 0.9054913224778073,
 '473905218555423969': 0.999215584070246,
 '238925310298938861': 0.9892820211598832,
 '9598871851189945502': 0.998740058266591,
 '5258378690008715588': 0.9964105738024341,
 '4688218153840595469': 0.9998807863822863,
 '0010295111715775250': 0.9976275979329572,
 '0842088345978971117': 0.9999895880848787,
 '7463172420271311409': 0.973894184905549,
 '8530723892127582602': 0.9999127893994905,
 '2144335436738770147': 0.9999814265888813,
 '9687976620185337812': 0.9995783876900877,
 '7455554400123317161': 0.9988817998069975,
 '2703137619338184529': 0.9987976301394139,
 '2766162501835274072': 0.9944232454603569,
 '7745913892709272663': 0.999919010767969,
 '9445171435422230535': 0.9998998104597683,
 '6684428411656994293': 0.999943660329656

In [80]:
sorted_dict = sorted(similarities.items(), key=lambda x: x[1])
sorted_dict[-5:]

[('8321566838784998459', 0.999988213949983),
 ('0842088345978971117', 0.9999895880848787),
 ('4062630166370650304', 0.9999930220712488),
 ('813207140579582070', 0.9999985949965018),
 ('7445235885559107095', 0.9999989174633006)]

In [81]:
recommendation(df,sorted_dict)

['Google 22 oz Water Bottle',
 "Google Women's Convertible Vest-Jacket Black",
 'Google Twill Cap']

In [92]:
bought_items = df.filter((pl.col('fullVisitorId') == '7483600664917507409') & (pl.col('transactionId') != 'null')).select(pl.col('v2ProductName')).unique()

In [95]:
for i in range(0,len(bought_items)):
    print(bought_items[i])

shape: (1, 1)
┌─────────────────────────────────┐
│ v2ProductName                   │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ Google Women's Short Sleeve Ba… │
└─────────────────────────────────┘
shape: (1, 1)
┌─────────────────────────────────┐
│ v2ProductName                   │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ Android 17oz Stainless Steel S… │
└─────────────────────────────────┘
shape: (1, 1)
┌─────────────────────────────────┐
│ v2ProductName                   │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ Google Men's Bike Short Sleeve… │
└─────────────────────────────────┘
shape: (1, 1)
┌─────────────────────────────────┐
│ v2ProductName                   │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ Google

In [59]:
# Need to split the data into subsets & perform the target encoding... k fold target encoding
# num_samples = len(new_df) // 8
# num_samples
# new_df = new_df.with_columns(pl.DataFrame({'kfold':np.repeat(np.arange(1, 9), num_samples)}))
# new_df

In [60]:
df.write_csv("../data/cleaned_google_analytics.csv")

### Data Pre-processing Pipeline

In [83]:
def preprocessing_pipeline(df: pl.DataFrame) -> pl.DataFrame:
    """
    Args:
        df : pl.DataFrame

    Returns:
        pl.DataFrames: pre-processed dataframe
    """

    df = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
    encoded_df = encoding(df)
    
    user_profiles = encoded_df.group_by(pl.col('fullVisitorId')).agg(
        pl.col('country'), pl.col('browser'),
        pl.col('transactionId'), pl.col('v2ProductName')
    )
    user_profiles_no_id = user_profiles.drop('fullVisitorId')
    user_profiles_no_id = user_profiles_no_id.to_numpy()
    active_users = encoded_df.drop('fullVisitorId')
    active_users = active_users.to_numpy()

    similarities = calculate_similarity(user_profiles, user_profiles_no_id, active_users)
    sorted_dict = sorted(similarities.items(), key=lambda x: x[1])

    
    df.write_csv("../data/cleaned_google_analytics.csv")
    return recommendation(df,sorted_dict) 

In [84]:
preprocessing_pipeline(df)

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklea

ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.

# Conclusions 
- 